In [1]:
import os

In [2]:
%pwd

'c:\\Users\\kk061\\OneDrive\\Desktop\\python\\Mlflow Main\\Heart Stroke Prediction\\notebook'

In [5]:
import pandas as pd
df=pd.read_csv('artifacts/data_transformation/train.csv')
df.isnull().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [3]:
os.chdir('../')
%pwd

'c:\\Users\\kk061\\OneDrive\\Desktop\\python\\Mlflow Main\\Heart Stroke Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str

In [5]:
from src.heartstrokeprediction.constants import *
from src.heartstrokeprediction.utils.common import read_yaml, create_directories

[2025-04-09 16:52:27,473: INFO: __init__: This is an info message.]
[2025-04-09 16:52:27,475: ERROR: __init__: This is an error message.]


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            target_column = schema.name
            
        )

        return model_trainer_config

In [7]:
import pandas as pd
import os
from src.heartstrokeprediction import logger
from sklearn.linear_model import LogisticRegression
import joblib
from sklearn.preprocessing import StandardScaler


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Load the training and testing data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Split features (X) and target (y) for both train and test datasets
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        # Standard scaling (feature normalization) - fit the scaler on the training set
        scaler = StandardScaler()
        train_x_scaled = scaler.fit_transform(train_x)  # Fit and transform the training data
        test_x_scaled = scaler.transform(test_x)  # Only transform the test data

        # Initialize and train the model (using Linear Regression for regression task)
        model = LogisticRegression(random_state=42)  # Replace with LinearRegression() if regression
        model.fit(train_x_scaled, train_y)

        # Predict on both train and test sets
        train_predictions = model.predict(train_x_scaled)
        test_predictions = model.predict(test_x_scaled)

        # Save the trained model and scaler
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        joblib.dump(scaler, os.path.join(self.config.root_dir, "scaler.pkl"))  # Save the scaler

        print(f"Model saved to {os.path.join(self.config.root_dir, self.config.model_name)}")
        print(f"Scaler saved to {os.path.join(self.config.root_dir, 'scaler.pkl')}")

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-04-09 16:52:35,431: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-04-09 16:52:35,436: INFO: common: YAML file: params.yaml loaded successfully]
[2025-04-09 16:52:35,440: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-04-09 16:52:35,445: INFO: common: created directory at: artifacts]
[2025-04-09 16:52:35,446: INFO: common: created directory at: artifacts/model_trainer]
Model saved to artifacts/model_trainer\model.joblib
Scaler saved to artifacts/model_trainer\scaler.pkl


c:\Users\kk061\OneDrive\Desktop\python\Mlflow Main\Heart Stroke Prediction\venv\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
